In [1]:
DOCKER_REGISTRY = 'index.docker.io/dejangolubovic'
my_namespace = 'dejan-golubovic'

In [2]:
num_chief = 1 #number of Chief in TFJob 
num_ps = 1  #number of PS in TFJob 
num_workers = 2  #number of Worker in TFJob 
model_dir = "/mnt"
export_path = "/mnt/export" 
train_steps = "1000"
batch_size = "100"
learning_rate = "0.01"
pv_name = 'pvc-f8cc766b-aef7-4456-99bb-8c99621094af'
pvc_name = 'dejan-fairing-pvc-2'

In [3]:
import uuid
from kubeflow import fairing   
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

tfjob_name = f'mnist-training-{uuid.uuid4().hex[:4]}'
print(tfjob_name)
output_map =  {
    "Dockerfile": "Dockerfile",
    "mnist.py": "mnist.py"
}

context_source = ''

command=["python",
         "/opt/mnist.py",
         "--tf-model-dir=" + model_dir,
         "--tf-export-dir=" + export_path,
         "--tf-train-steps=" + train_steps,
         "--tf-batch-size=" + batch_size,
         "--tf-learning-rate=" + learning_rate]

minio_context_source = MinioContextSource(
            endpoint_url='http://minio-service.kubeflow:9000/',
            minio_secret='minio',
            minio_secret_key='minio123',#
            region_name='region')

fairing.config.set_preprocessor('python', command=command, path_prefix="/app", output_map=output_map)
fairing.config.set_builder(name='cluster', registry=DOCKER_REGISTRY, \
                           context_source=minio_context_source, cleanup=True, \
                          pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.set_deployer(name='tfjob', namespace=my_namespace, stream_log=False, job_name=tfjob_name,
                            chief_count=num_chief, worker_count=num_workers, ps_count=num_ps, 
                            pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.run()
#fairing.config.run()

[W 200917 15:45:40 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[W 200917 15:45:40 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[I 200917 15:45:40 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7f389557cf98>
[I 200917 15:45:40 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7f386bf65c18>
[I 200917 15:45:40 config:138] Using deployer: <kubeflow.fairing.deployers.tfjob.tfjob.TfJob object at 0x7f386c1a8c18>
[I 200917 15:45:40 cluster:46] Building image using cluster builder.
[I 200917 15:45:40 base:107] Creating docker context: /tmp/fairing_context_z_bzlpq3
[W 200917 15:45:40 base:94] /tmp/fairing_dockerfile_daq4undv already exists in Fairing context, skipping...


mnist-training-565f


[W 200917 15:45:40 manager:298] Waiting for fairing-builder-44vf6-qbhjr to start...
[W 200917 15:45:40 manager:298] Waiting for fairing-builder-44vf6-qbhjr to start...
[W 200917 15:45:40 manager:298] Waiting for fairing-builder-44vf6-qbhjr to start...
[I 200917 15:45:56 manager:304] Pod started running True


INFO[0002] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0004] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0006] Built cross stage deps: map[]
INFO[0006] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0008] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0010] Executing 0 build triggers
INFO[0010] Unpacking rootfs as cmd ADD mnist.py /opt/mnist.py requires it.
INFO[0033] Taking snapshot of full filesystem...
INFO[0034] Resolving 27181 paths
INFO[0038] Using files from context: [/kaniko/buildcontext/mnist.py]
INFO[0038] ADD mnist.py /opt/mnist.py
INFO[0038] RUN chmod +x /opt/mnist.py
INFO[0038] cmd: /bin/sh
INFO[0038] args: [-c chmod +x /opt/mnist.py]
INFO[0038] Running: [/bin/sh -c chmod +x /opt/mnist.py]
INFO[0038] ENTRYPOINT ["/usr/bin/python"]
INFO[0038] CMD ["/opt/mnist.py"]
INFO[0038] Taking snapshot of full filesystem...
INFO[0038] Resolving 27182 paths


[W 200917 15:46:44 cluster:106] Cleaning up job fairing-builder-44vf6...
[W 200917 15:46:44 job:101] The tfjob mnist-training-565f launched.


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7f389557cf98>,
 <kubeflow.fairing.deployers.tfjob.tfjob.TfJob at 0x7f386c1a8c18>)

In [6]:
tfjob_name = 'mnist-training-565f'
tfjob_namespace = 'dejan-golubovic'

In [4]:
from kubeflow.tfjob import TFJobClient
tfjob_client = TFJobClient()

tfjob_client.get(tfjob_namespace, namespace=tfjob_namespace)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'TFJob',
 'metadata': {'creationTimestamp': '2020-09-17T15:46:44Z',
  'generateName': 'fairing-tfjob-',
  'generation': 1,
  'labels': {'fairing-deployer': 'tfjob',
   'fairing-id': 'fce16af2-f8fc-11ea-979e-0a580a640d4a'},
  'name': 'mnist-training-565f',
  'namespace': 'dejan-golubovic',
  'resourceVersion': '12916877',
  'selfLink': '/apis/kubeflow.org/v1/namespaces/dejan-golubovic/tfjobs/mnist-training-565f',
  'uid': '9842cdda-e047-42d0-b2e2-ece37d0e0823'},
 'spec': {'tfReplicaSpecs': {'Chief': {'replicas': 1,
    'template': {'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
      'labels': {'fairing-deployer': 'tfjob',
       'fairing-id': 'fce16af2-f8fc-11ea-979e-0a580a640d4a'},
      'name': 'fairing-deployer'},
     'spec': {'containers': [{'command': ['python',
         '/opt/mnist.py',
         '--tf-model-dir=/mnt',
         '--tf-export-dir=/mnt/export',
         '--tf-train-steps=1000',
         '--tf-batch-size=1

In [5]:
tfjob_client.wait_for_job(tfjob_names, namespace=tfjob_namespace, watch=True)

NAME                           STATE                TIME                          
mnist-training-565f            Succeeded            2020-09-17T15:48:59Z          


In [7]:
tfjob_client.is_job_succeeded(tfjob_name, namespace=tfjob_namespace)

True

In [8]:
tfjob_client.get_logs(tfjob_name, namespace=tfjob_namespace)

[I 200917 15:52:00 tf_job_client:386] The logs of Pod mnist-training-565f-chief-0:
    
    
    W0917 15:47:31.123248 140313312196416 module_wrapper.py:139] From /opt/mnist.py:155: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.
    
    
    W0917 15:47:31.123630 140313312196416 module_wrapper.py:139] From /opt/mnist.py:155: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.
    
    
    W0917 15:47:31.125045 140313312196416 module_wrapper.py:139] From /opt/mnist.py:160: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.
    
    INFO:tensorflow:TF_CONFIG {"cluster":{"chief":["mnist-training-565f-chief-0.dejan-golubovic.svc:2222"],"ps":["mnist-training-565f-ps-0.dejan-golubovic.svc:2222"],"worker":["mnist-training-565f-worker-0.dejan-golubovic.svc:2222","mnist-training-565f-worker-1.dejan-golubovic.svc:2222"]},"task":{"type":"chief","index":0},"environment":"

In [9]:
from kubeflow.fairing.deployers.kfserving.kfserving import KFServing
isvc_name = 'dejan-mnist-service'
isvc = KFServing('tensorflow', namespace=tfjob_namespace, isvc_name=isvc_name,
                 default_storage_uri='pvc://' + pvc_name + '/export')
isvc.deploy(isvc.generate_isvc())

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
dejan-mnist-service  Unknown                                                                                      
dejan-mnist-service  False                                                                                        
dejan-mnist-service  False                                                                                        
dejan-mnist-service  False                                                                                        
dejan-mnist-service  False                                                                                        
dejan-mnist-service  True       100                             http://dejan-mnist-service.dejan-golubovic.exam...


[I 200917 15:54:31 kfserving:127] Deployed the InferenceService dejan-mnist-service successfully.


'dejan-mnist-service'

In [10]:
from kfserving import KFServingClient
kfserving_client = KFServingClient()
kfserving_client.get(namespace=tfjob_namespace)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'items': [{'apiVersion': 'serving.kubeflow.org/v1alpha2',
   'kind': 'InferenceService',
   'metadata': {'creationTimestamp': '2020-09-17T15:53:54Z',
    'generateName': 'fairing-kfserving-',
    'generation': 1,
    'name': 'dejan-mnist-service',
    'namespace': 'dejan-golubovic',
    'resourceVersion': '12920604',
    'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/dejan-golubovic/inferenceservices/dejan-mnist-service',
    'uid': 'cdccdcb3-21f3-4c7e-bc0b-fb2bd7a31f49'},
   'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
         'memory': '2Gi'},
        'requests': {'cpu': '1', 'memory': '2Gi'}},
       'runtimeVersion': '1.14.0',
       'storageUri': 'pvc://dejan-fairing-pvc-2/export'}}}},
   'status': {'address': {'url': 'http://dejan-mnist-service.dejan-golubovic.svc.cluster.local/v1/models/dejan-mnist-service:predict'},
    'canary': {},
    'conditions': [{'lastTransitionTime':

In [11]:
mnist_isvc = kfserving_client.get(isvc_name, namespace=tfjob_namespace)
mnist_isvc_name = mnist_isvc['metadata']['name']
mnist_isvc_endpoint = mnist_isvc['status'].get('url', '')
print("MNIST Service Endpoint: " + mnist_isvc_endpoint)

MNIST Service Endpoint: http://dejan-mnist-service.dejan-golubovic.example.com/v1/models/dejan-mnist-service


In [28]:
#ISTIO_CLUSTER_IP=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.clusterIP}'
#CLUSTER_IP=ISTIO_CLUSTER_IP[0]
CLUSTER_IP = '10.254.73.13'
MODEL_HOST=f"Host: {mnist_isvc_name}.{tfjob_namespace}.example.com"
print(MODEL_HOST)
print('curl -v -H'+MODEL_HOST+' http://'+CLUSTER_IP+'/v1/models/'+mnist_isvc_name+':predict -d @./input.json')
!curl -v -H "{MODEL_HOST}" http://{CLUSTER_IP}/v1/models/{mnist_isvc_name}:predict -d @./input.json

Host: dejan-mnist-service.dejan-golubovic.example.com
curl -v -HHost: dejan-mnist-service.dejan-golubovic.example.com http://10.254.73.13/v1/models/dejan-mnist-service:predict -d @./input.json
*   Trying 10.254.73.13...
* TCP_NODELAY set
* Connected to 10.254.73.13 (10.254.73.13) port 80 (#0)
> POST /v1/models/dejan-mnist-service:predict HTTP/1.1
> Host: dejan-mnist-service.dejan-golubovic.example.com
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Length: 2052
> Content-Type: application/x-www-form-urlencoded
> Expect: 100-continue
> 
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
< HTTP/1.1 503 Service Unavailable
< content-length: 91
< content-type: text/plain
< date: Thu, 17 Sep 2020 16:04:47 GMT
< server: envoy
< 
* Connection #0 to host 10.254.73.13 left intact
upstream connect error or disconnect/reset before headers. reset reason: connection failure